In [100]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient

In [101]:
# I need to update the documents, to add a field called location, that includes the point type and coordinates for each document.


# Connect to the MongoDB server
client = MongoClient("localhost:27017")
db = client['geospatial']

# Select the database and collection


# Update all documents to add the geometry field
dict_of_places = {'bar':13003,
                   'physiotherapy':15026,
                   'social_club':12097,
                   'tennis_store':17130,
                   'residential':12094,
                   'park': 16032,
                   'university': 12013,
                   'gym':18021,
                   'parking':19020,
                   'public_transport':19054,
                   'tennis_court':18047
                    }
# I loop through all the names of my collections in mongo.
for key in dict_of_places.keys():
    collection = db[key]
    for document in collection.find():
        lat = document["geocodes"]["main"]['latitude']
        lng = document["geocodes"]["main"]['longitude']
        geometry = {
            "type": "Point",
            "coordinates": [lng, lat]
        }
        collection.update_one({"_id": document["_id"]}, {"$set": {"geometry": geometry}})

In [102]:
client = MongoClient("localhost:27017")
db = client['geospatial']

In [103]:
districts = db.get_collection('districts_full_equip')

In [104]:
# I am going to loop through all the documents of each collection, and see what is the district containing the highest number of places.
def what_district (location):
    what_district = districts.find_one(
                                {"geometry":
                                    {"$geoIntersects": 
                                        {"$geometry": 
                                            location}}},
                            projection = {'abb_name':1})['abb_name']
    return what_district

In [133]:
list_of_results = []
for category in dict_of_places.keys():
    collection=db[category]
  
    for local in collection.find():
        new_list = []
        geometry=local['geometry']
        name = local['name']
        coordinates = geometry['coordinates']
        district_found = what_district(geometry)
        print(district_found)
        new_list = [name,category,geometry,district_found]
        list_of_results.append(new_list)
       
    

Sydney
Inner West
Sydney
Sydney
Sydney
Sydney
Sydney
Sydney
Sydney
Inner West
Woollahra
Randwick
Sydney
Inner West
Sydney
Inner West
Waverley
Burwood
Sydney
Sydney
Sydney
Sydney
Sydney
Inner West
Sydney
Willoughby
Unincorporated - Sydney Harbour Area
Sydney
Bayside
Sydney
Ryde
Fairfield
Hornsby
Canada Bay
Burwood
Canterbury-Bankstown
Inner West
Inner West
Inner West
Sydney
North Sydney
North Sydney
Sydney
Sydney
Sydney
Bayside
Sydney
Sydney
Waverley
Sutherland
Penrith
Parramatta
Cumberland
Parramatta
Parramatta
Ryde
The Hills
Canterbury-Bankstown
Canterbury-Bankstown
Hornsby
Unincorporated - Sydney Harbour Area
Canada Bay
Inner West
Northern Beaches
The Hills
Northern Beaches
Parramatta
Canada Bay
Waverley
Ryde
Fairfield
Parramatta
Cumberland
Sutherland
Blacktown
Penrith
Ryde
Northern Beaches
Canterbury-Bankstown
Burwood
Fairfield
Canterbury-Bankstown
Willoughby
Inner West
Georges River
Sydney
Georges River
Sydney
Georges River
Sydney
Ryde
Cumberland
Inner West
Willoughby
Lane Cove
Par

In [135]:
list_of_results
df_results=pd.DataFrame(list_of_results)
df_results

,0,1,2,3
0,Opera Bar,bar,"{'type': 'Point', 'coordinates': [151.214458, ...",Sydney
1,Wayward Brewing Co.,bar,"{'type': 'Point', 'coordinates': [151.174816, ...",Inner West
2,The Potting Shed at the Grounds,bar,"{'type': 'Point', 'coordinates': [151.194181, ...",Sydney
3,The Wild Rover,bar,"{'type': 'Point', 'coordinates': [151.209629, ...",Sydney
4,The Grounds of Alexandria,bar,"{'type': 'Point', 'coordinates': [151.194368, ...",Sydney
...,...,...,...,...
300,Crown Tennis Court,tennis_court,"{'type': 'Point', 'coordinates': [151.201158, ...",Sydney
301,Kent Street Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.203822, ...",Sydney
302,Darling South Ping Pong,tennis_court,"{'type': 'Point', 'coordinates': [151.202698, ...",Sydney
303,Sydney Park Village Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.185803, ...",Sydney


In [136]:
df_results.columns=[['name','category','coordinates','district']]

In [142]:
df_results

,name,category,coordinates,district
0,Opera Bar,bar,"{'type': 'Point', 'coordinates': [151.214458, ...",Sydney
1,Wayward Brewing Co.,bar,"{'type': 'Point', 'coordinates': [151.174816, ...",Inner West
2,The Potting Shed at the Grounds,bar,"{'type': 'Point', 'coordinates': [151.194181, ...",Sydney
3,The Wild Rover,bar,"{'type': 'Point', 'coordinates': [151.209629, ...",Sydney
4,The Grounds of Alexandria,bar,"{'type': 'Point', 'coordinates': [151.194368, ...",Sydney
...,...,...,...,...
300,Crown Tennis Court,tennis_court,"{'type': 'Point', 'coordinates': [151.201158, ...",Sydney
301,Kent Street Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.203822, ...",Sydney
302,Darling South Ping Pong,tennis_court,"{'type': 'Point', 'coordinates': [151.202698, ...",Sydney
303,Sydney Park Village Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.185803, ...",Sydney


In [154]:
df_results['district']=df_results['district']

In [155]:
df_results

,name,category,coordinates,district,cagtegory
0,Opera Bar,bar,"{'type': 'Point', 'coordinates': [151.214458, ...",Sydney,bar
1,Wayward Brewing Co.,bar,"{'type': 'Point', 'coordinates': [151.174816, ...",Inner West,bar
2,The Potting Shed at the Grounds,bar,"{'type': 'Point', 'coordinates': [151.194181, ...",Sydney,bar
3,The Wild Rover,bar,"{'type': 'Point', 'coordinates': [151.209629, ...",Sydney,bar
4,The Grounds of Alexandria,bar,"{'type': 'Point', 'coordinates': [151.194368, ...",Sydney,bar
...,...,...,...,...,...
300,Crown Tennis Court,tennis_court,"{'type': 'Point', 'coordinates': [151.201158, ...",Sydney,tennis_court
301,Kent Street Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.203822, ...",Sydney,tennis_court
302,Darling South Ping Pong,tennis_court,"{'type': 'Point', 'coordinates': [151.202698, ...",Sydney,tennis_court
303,Sydney Park Village Tennis Courts,tennis_court,"{'type': 'Point', 'coordinates': [151.185803, ...",Sydney,tennis_court


In [166]:
df_dict = {
            'name':[df_results['name']],
            'category': [df_results['category']],
            'coordinates': [df_results['coordinates']],
            'district': [df_results['district']]
                                                        }


In [167]:
df_porfa = pd.DataFrame(df_dict)

In [168]:
df_porfa

,name,category,coordinates,district
0,name 0 ...,category 0 bar 1 ...,coo...,district 0 Sydney 1 Inn...
